---
title: "Assignment 2 - Solutions"
bibliography: ../reading_list.bib
format:
  html:
    toc: true
    toc-depth: 2
    toc-location: left
    theme: default
    code-fold: false
    code-tools: true
    embed-resources: true
    self-contained: true
---

In [ ]:
#| output: false
using LinearAlgebra, Plots, Distributions

# Question 1

Using the same model outlined in [recitation 2](../recitations/recitation-2.qmd) and [recitation 3](../recitations/recitation-3.qmd), solve two versions of the model with $\rho = 0.95$ and $\rho=0.1$, keeping all other parameters the same as in those recitations.

## Solution

First, let's set up all the functions we need from the recitations:


In [ ]:
# Utility function
utility(c,σ) = c^(1 - σ) / (1 - σ)

# Tauchen method for discretizing AR(1) process
function tauchen(N, ρ, σ)
    # Unconditional standard deviation of the process
    σ_y = σ / sqrt(1 - ρ^2)

    # Create the grid
    y_max = 3. * σ_y
    y_min = - 3. * σ_y
    grid = range(y_min, y_max, length=N)

    # Step size
    step = (y_max - y_min) / (N - 1)

    # Initialize transition matrix
    Π = zeros(N, N)

    # Standard normal distribution
    d = Normal(0, 1)

    # Fill in the transition matrix
    for i in 1:N
        for j in 1:N
            if j == 1
                # Probability of transitioning to the lowest state
                Π[j, i] = cdf(d, (grid[j] - ρ * grid[i] + step/2) / σ)
            elseif j == N
                # Probability of transitioning to the highest state
                Π[j, i] = 1 - cdf(d, (grid[j] - ρ * grid[i] - step/2) / σ)
            else
                # Probability of transitioning to intermediate states
                Π[j, i] = cdf(d, (grid[j] - ρ * grid[i] + step/2) / σ) -
                         cdf(d, (grid[j] - ρ * grid[i] - step/2) / σ)
            end
        end
    end
    return collect(grid), Π
end

# Solve value function at a point
function solve_value(ai,yi,V,p,t)
    (;σ,β,Π,r,asset_grid,income_grid,net_income) = p
    a = asset_grid[ai]
    y = exp(income_grid[yi,t])
    y_net = net_income(y)
    vmax = -Inf
    amax = 0
    for ai_next in eachindex(asset_grid)
        a_next = asset_grid[ai_next]
        c = y_net + a - a_next/(1+r)
        if c>0
            @views v = utility(c,σ) + β*dot(Π[:,yi],V[:,ai_next,t+1])
            if v>vmax
                vmax = v
                amax = a_next
            end
        end
    end
    return vmax,amax
end

# Iterate value function
function iterate_value_function!(p,V,A,t)
    for ai in axes(V,2), yi in axes(V,1)
        v,a_next = solve_value(ai,yi,V,p,t)
        V[yi,ai,t] = v
        A[yi,ai,t] = a_next
    end
end

# Solve the model
function solve_model(p)
    (;T,Π,asset_grid) = p
    K_a = length(asset_grid)
    K_y = size(Π,1)
    V = zeros(K_y,K_a,T+1)
    A = zeros(K_y,K_a,T)
    for t in reverse(1:T)
        iterate_value_function!(p,V,A,t)
    end
    return (;V,A)
end

# No tax function
no_tax(x) = x

Now solve for $\rho = 0.95$:


In [ ]:
μ = log(10.)
T = 60

# Model with ρ = 0.95
egrid_95, Π_95 = tauchen(5, 0.95, 0.1)
income_grid_95 = [μ + egrid_95[i] for i in 1:5, t in 1:T]

p_95 = (;
    T,
    asset_grid = LinRange(0, 100, 100),
    income_grid = income_grid_95,
    β = 0.96,
    r = 0.04,
    Π = Π_95,
    σ = 2.,
    net_income = no_tax
)

model_95 = solve_model(p_95)

Now solve for $\rho = 0.1$:


In [ ]:
# Model with ρ = 0.1
egrid_10, Π_10 = tauchen(5, 0.1, 0.1)
income_grid_10 = [μ + egrid_10[i] for i in 1:5, t in 1:T]

p_10 = (;
    T,
    asset_grid = LinRange(0, 100, 100),
    income_grid = income_grid_10,
    β = 0.96,
    r = 0.04,
    Π = Π_10,
    σ = 2.,
    net_income = no_tax
)

model_10 = solve_model(p_10)

Let's compare the value functions:


In [ ]:
plot(model_95.V[:,1,1], label = "ρ = 0.95",
     xlabel = "Income state", ylabel = "Value",
     title = "Value Function Comparison (a=0, t=1)")
plot!(model_10.V[:,1,1], label = "ρ = 0.1")

With lower persistence ($\rho = 0.1$), the value function is higher across all income states because shocks are less persistent.

# Question 2

Given agents live for $T$ periods, note that for a guaranteed consumption level $\bar{c}$, we have:
$$ \mathcal{V}(\bar{c}) = \sum_{t=0}^{T}\beta^{t}u(\bar{c}) = \frac{1-\beta^{T}}{1-\beta}u(\bar{c}).$$

Given this, show that:
$$ \bar{c}(y) = \left[(1-\sigma)\frac{1-\beta}{1-\beta^{T}}V(0,y)\right]^{1/(1-\sigma)} $$
is the consumption equivalent that, if delivered with certainty in every period, would achieve the same value as $V(0,y)$.

## Solution

By definition, the consumption equivalent $\bar{c}(y)$ satisfies:
$$ \mathcal{V}(\bar{c}(y)) = V(0,y) $$

Substituting the formula for $\mathcal{V}$:
$$ \frac{1-\beta^{T}}{1-\beta}u(\bar{c}(y)) = V(0,y) $$

For CRRA utility, $u(c) = \frac{c^{1-\sigma}}{1-\sigma}$, so:
$$ \frac{1-\beta^{T}}{1-\beta} \cdot \frac{\bar{c}(y)^{1-\sigma}}{1-\sigma} = V(0,y) $$

Solving for $\bar{c}(y)$:
$$ \bar{c}(y)^{1-\sigma} = V(0,y) \cdot \frac{(1-\sigma)(1-\beta)}{1-\beta^{T}} $$

Taking the $\frac{1}{1-\sigma}$ power of both sides:
$$ \bar{c}(y) = \left[(1-\sigma)\frac{1-\beta}{1-\beta^{T}}V(0,y)\right]^{1/(1-\sigma)} $$

# Question 3

Assume that the government now levies a 10% proportional income tax that it then redistributes as a lump sum. Thus, net income for an agent is
$$ \tilde{y} = 0.9 \times y + 0.1\times\mathbb{E}[y] $$
where $\mathbb{E}[y] = \sum_{y}\pi^*(y)y$ and $\pi^*$ is the stationary distribution of the transition matrix $\Pi$. Write code to solve the model with this system of taxes and transfers. Show that this net income function respects the aggregate resource constraint.

## Solution

First, we need helper functions to find the stationary distribution and construct the tax system:


In [ ]:
# Find stationary distribution
function solve_steady_state(Π)
    K = size(Π, 2)
    vals, e = eigen(Π)
    return e[:, K] ./ sum(e[:, K])
end

# Construct proportional tax with lump-sum transfer
function proportional_tax_transfer(y, mean_y, τ)
    return (1 - τ) * y + τ * mean_y
end

Now solve the model with 10% proportional tax for $\rho = 0.95$:


In [ ]:
# Calculate stationary distribution and mean income
pi_star_95 = solve_steady_state(Π_95)
mean_y_95 = dot(pi_star_95, exp.(income_grid_95[:, 1]))

# Tax rate
τ = 0.1

# Create net income function
net_income_tax = y -> proportional_tax_transfer(y, mean_y_95, τ)

# Solve model with taxes
p_95_tax = (;p_95..., net_income = net_income_tax)
model_95_tax = solve_model(p_95_tax)

**Showing the resource constraint is respected:**

The aggregate resource constraint requires that average net income equals average gross income. The resource constraint holds because:
$$\mathbb{E}[\tilde{y}] = \mathbb{E}[0.9y + 0.1\mathbb{E}[y]] = 0.9\mathbb{E}[y] + 0.1\mathbb{E}[y] = \mathbb{E}[y]$$

Now solve for $\rho = 0.1$ with taxes:


In [ ]:
# Calculate stationary distribution and mean income for ρ = 0.1
pi_star_10 = solve_steady_state(Π_10)
mean_y_10 = dot(pi_star_10, exp.(income_grid_10[:, 1]))

# Create net income function
net_income_tax_10 = y -> proportional_tax_transfer(y, mean_y_10, τ)

# Solve model with taxes
p_10_tax = (;p_10..., net_income = net_income_tax_10)
model_10_tax = solve_model(p_10_tax)

# Question 4

Define
$$W^{r} = \sum_{y}\pi^*(y)V^{r}(0,y) = \mathbb{E}[V^{r}(0,y)]$$
to be utilitarian welfare under regime $r\in\{A,B\}$ where regime $A$ is the laissez-faire economy and regime $B$ is the economy with taxes and transfers. For both models, calculate the decomposition of the consumption equivalent measure of welfare gains.

## Solution

We'll calculate the welfare decomposition for both $\rho = 0.95$ and $\rho = 0.1$.

### For $\rho = 0.95$:


In [ ]:
(;σ, β) = p_95

# Aggregate consumption (same in both regimes)
C_95 = dot(pi_star_95, exp.(income_grid_95[:, 1]))

# Certainty equivalent function
V_cert(x) = (1 - β^T) / (1 - β) * utility(x, σ)

# Consumption equivalents for regime A (no tax)
cbar_95_A = ((1 - σ) * (1 - β) / (1 - β^T) * model_95.V[:, 1, 1]) .^ (1 / (1 - σ))

# Consumption equivalents for regime B (with tax)
cbar_95_B = ((1 - σ) * (1 - β) / (1 - β^T) * model_95_tax.V[:, 1, 1]) .^ (1 / (1 - σ))

# Average consumption equivalents
Cbar_95_A = dot(pi_star_95, cbar_95_A)
Cbar_95_B = dot(pi_star_95, cbar_95_B)

# Utilitarian welfare
W_95_A = dot(pi_star_95, model_95.V[:, 1, 1])
W_95_B = dot(pi_star_95, model_95_tax.V[:, 1, 1])

# Welfare decomposition
ω_95 = (W_95_B / W_95_A)^(1 / (1 - σ)) - 1  # Total gain
γ_95 = (V_cert(Cbar_95_B) / V_cert(Cbar_95_A))^(1 / (1 - σ)) - 1  # Insurance gain
α_95 = ((W_95_B / V_cert(Cbar_95_B)) / (W_95_A / V_cert(Cbar_95_A)))^(1 / (1 - σ)) - 1  # Redistribution gain

println("ρ = 0.95 Welfare Decomposition:")
println("Total gain (1+ω): ", 1 + ω_95)
println("Insurance gain (1+γ): ", 1 + γ_95)
println("Redistribution gain (1+α): ", 1 + α_95)
println("Verification (1+ω) ≈ (1+γ)(1+α): ", (1 + γ_95) * (1 + α_95))

### For $\rho = 0.1$:


In [ ]:
# Aggregate consumption (same in both regimes)
C_10 = dot(pi_star_10, exp.(income_grid_10[:, 1]))

# Consumption equivalents for regime A (no tax)
cbar_10_A = ((1 - σ) * (1 - β) / (1 - β^T) * model_10.V[:, 1, 1]) .^ (1 / (1 - σ))

# Consumption equivalents for regime B (with tax)
cbar_10_B = ((1 - σ) * (1 - β) / (1 - β^T) * model_10_tax.V[:, 1, 1]) .^ (1 / (1 - σ))

# Average consumption equivalents
Cbar_10_A = dot(pi_star_10, cbar_10_A)
Cbar_10_B = dot(pi_star_10, cbar_10_B)

# Utilitarian welfare
W_10_A = dot(pi_star_10, model_10.V[:, 1, 1])
W_10_B = dot(pi_star_10, model_10_tax.V[:, 1, 1])

# Welfare decomposition
ω_10 = (W_10_B / W_10_A)^(1 / (1 - σ)) - 1  # Total gain
γ_10 = (V_cert(Cbar_10_B) / V_cert(Cbar_10_A))^(1 / (1 - σ)) - 1  # Insurance gain
α_10 = ((W_10_B / V_cert(Cbar_10_B)) / (W_10_A / V_cert(Cbar_10_A)))^(1 / (1 - σ)) - 1  # Redistribution gain

println("ρ = 0.1 Welfare Decomposition:")
println("Total gain (1+ω): ", 1 + ω_10)
println("Insurance gain (1+γ): ", 1 + γ_10)
println("Redistribution gain (1+α): ", 1 + α_10)
println("Verification (1+ω) ≈ (1+γ)(1+α): ", (1 + γ_10) * (1 + α_10))

# Question 5

Comment on the difference in this decomposition for $\rho=0.95$ vs $\rho=0.1$. Can you give some intuition for the difference?

## Solution

Let's create a comparison table:


In [ ]:
using Printf

println("\nComparison of Welfare Decompositions:")
println("="^60)
println(@sprintf("%-20s %12s %12s", "Component", "ρ = 0.95", "ρ = 0.1"))
println("="^60)
println(@sprintf("%-20s %12.4f %12.4f", "Total (1+ω)", 1 + ω_95, 1 + ω_10))
println(@sprintf("%-20s %12.4f %12.4f", "Insurance (1+γ)", 1 + γ_95, 1 + γ_10))
println(@sprintf("%-20s %12.4f %12.4f", "Redistribution (1+α)", 1 + α_95, 1 + α_10))
println("="^60)

**Intuition:**

- High persistence ($\rho \approx 1$) → income inequality is mostly *permanent* → welfare gains mostly redistribution
- Low persistence ($\rho \approx 0$) → income inequality is mostly *transitory* → welfare gains mostly insurance
- When persistence ($\rho$) is lower, shocks are easier to insure against → lower overall welfare gains.
